In [4]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 6610
# Algorithms

## Computational Complexity and NP-Hardness


Going back to our discussions of asymptotic complexity, we said that we were interested in algorithms for computational problems that:

- Did polynomial work in the input size

- Leveraged concurrency to achieve span (i.e., parallel speedup)

We have studied different algorithmic paradigms to try and achieve these two goals. But, when is this actually possible?

What problems are solvable with polynomial work? Of those, which allow us to achieve a good parallel speedup?

Or, which problems **aren't** solvable with polynomial work? Perhaps we could just avoid or approximate these, instead of trying to find efficient algorithms.


Why would we care about characterizing the difficulty of problems? Why not just do the best we can?

Given a problem $\mathcal{X}$ that we want to solve efficiently, we might not be able come up with a polynomial-work algorithm.

<a href="https://www.ac.tuwien.ac.at/people/szeider/cartoon/"><img src ="npc_cartoon1.png" width=70%></a>


The field of **computational complexity** tries to characterize problems by resource complexity (e.g., work, span, space).

We'll define some basic complexity classes and look at how much progress has been made in the last 50 years or so. 

In our discussion, we will look at *decision problems*, i.e., computational problems with YES/NO answers. 

Note that this is not a big restriction, because if we can solve a decision problem we can solve the optimization version using binary search.

Example: Consider a decision version of MST. Does a graph $G$ have an spanning tree of cost at most $k$? 

We can find an MST by doing binary search on $k$ (starting with say the sum of all edge costs). This requires times polynomial in the input. Why? 



For MST, we gave several very efficient algorithms (with work nearly linear in the input size). 

Are there any problems for which we haven't seen efficient algorithms?



Yes - TSP and Knapsack. Are there more efficient algorithms possible? Slightly, but the best we can do is essentially some kind of inefficient/exhaustive search over the solution space.

To justify such inefficient approaches, we'd like to give a superpolynomial lower bounds for TSP or Knapsack (or any problem for which we can't find an efficient algorithm).


<a href="https://www.ac.tuwien.ac.at/people/szeider/cartoon/"><img src ="npc_cartoon2.png" width=70%></a>

There are a number of very common/useful problems for which we can't seem to a) come up with good algorithms, or b) prove that no good algorithms exist.

Two more examples:

- Satisfiability ($\mathcal{3SAT}$): Given a logical formula $F(x_1, \ldots, x_n)$ that is written as a 3CNF ("CNF" stands for *conjunctive normal form*), is there an assignment of $x_1, \ldots, x_n$ such that $F$ yields evaluates to True?

- Independent Set ($\mathcal{IS}$): Given a graph $G=(V, E)$, is there a set $X\subseteq V$ of size at least $k$ such that no vertices $x, y\in X$ share an edge?

So what do we do?

### Checking versus Solving ###

Another interesting thing about these problems that we can't seem to solve is that, given a solution, we can actually check whether or not the solution is correct very quickly. We just don't know how to produce a "YES" instance efficiently.

For example, if we are give a truth assignment for a formula $F$, we can easily plug it in and check it. Likewise we can easily check solutions to TSP, Knapsack, Independent Set with polynomial work. 

The class of problems for which we can **check**, with a provided candidate solution, whether the input produces a YES answer is known as $\mathcal{NP}$. $\mathcal{P}$ is the class of problems for which we can compute solutions directly with polynomial work.

Now we know that $\mathcal{P}\subseteq\mathcal{NP}$, since we can efficiently check a problem solution by constructing it in polynomial time.

But does $\mathcal{P} = \mathcal{NP}$? Or more informally, do we need substantially more work to solve a problem than to check a solution to it?



### Reductions ###

Interestingly, many of these problems reduce to one another.

Recall that a problem $\mathcal{X}$ is *polynomial-time reducible* to a problem $\mathcal{Y}$ if we can i) perform an input transformation from $\mathcal{X}$ to $\mathcal{Y}$ and ii) an output transformation from $\mathcal{Y}$ to $\mathcal{X}$ with polynomial work. This shows that $\mathcal{Y}$ "is as hard" as $\mathcal{X}$, because an algorithm for $\mathcal{Y}$ then yields an algorithm for $\mathcal{X}$ (with an additional polynomial amount of work).

Example: $\mathit{3SAT} \leq \mathit{IS}$. 

Any logical formula can be written as a 3CNF, which has clauses AND'ed together with each clause OR'ing 3 literals. For example: $$ F(x_1, x_2, x_3, x_4) = (x_1 \vee x_2 \vee \neg{x_3}) \wedge (x_2 \vee x_3 \vee \neg{x_4})\wedge (x_1 \vee \neg{x_2} \vee x_4).$$ If $F$ is an input instance to $\mathit{3SAT}$, then we want to know if there is any assignment of $x_1, x_2, x_3, x_4$ that evaluates to TRUE.

To show that $\mathit{3SAT} \leq \mathit{IS}$, we need to show that if we can solve Independent Set, we can solve 3SAT. So given a logical formula $F$, how do we create a graph in which an independent set tells us about the satisfiability of clauses in $F$?

For a formula $F$ with $n$ variables and $k$ clauses, we create a graph $G_F=(V, E)$ so that $V$ has one vertex per literal in $F$. We construct $E$ such that all literals in a clause have edges between them, and every literals and its negation have an edge. Why is this an efficient input transformation?  

<img src="3SAT_IS.png" width = 75%>



An independent set $X$ in $G_F$ corresponds to a set of literals in $F$. Notice that the maximum size of an independent set in $G_F$ is $k$, corresponding to at most one literal per clause. Moreover, we will never choose a pair of vertices in the independent set with opposite literals since they are connected by an edge. Thus if we set all literals corresponding to vertices in $X$ to TRUE, we will satisfy $|X|$ clauses. So, $F$ is satisfiable if and only if there is an independent set of size $k$ in $G_F$.

<img src="3SAT_IS.png" width = 75%>

So what's the big deal if one problem in $\mathcal{NP}$ reduces to another?

### $\mathcal{P} = \mathcal{NP}$ and NP-Completeness ###

Leonid Levin and Steve Cook showed in the early 1970s that for any problem $\mathcal{X}\in\mathcal{NP}$, $\mathcal{X} \leq \mathit{SAT}$. So, $\mathit{3SAT}$ is **$\mathcal{NP}$-complete**.

If we could demonstrate a lower bound that $\mathit{3SAT}$ requires super-polynomial work, then we would prove that $\mathcal{P} \neq \mathcal{NP}$. If we came up with a polynomial work algorithm for $\mathit{3SAT}$, then we would prove that $\mathcal{P} = \mathcal{NP}$.

But we haven't been able to do either of these things, so far.

Does our reduction from $\mathit{3SAT}$ to $\mathit{IS}$ tell us anything interesting?


That Independent Set is $\mathcal{NP}$-complete!

What does this have to do with TSP or Knapsack?

After Levin and Cook showed the satisfiability problem was $\mathcal{NP}-complete$, Richard Karp used reductions to show 21 different problems were all $\mathcal{NP}$-complete.

<a href="http://cgi.di.uoa.gr/~sgk/teaching/grad/handouts/karp.pdf"><img src ="karp_21_problems.jpg" width=70%></a>

$\mathcal{NP}$-completeness doesn't provide us with concrete answers but it does help us understand the relative difficulty of problems. We currently have [many thousands](http://www.csc.kth.se/~viggo/problemlist/compendium.html) of such problems!

<a href="https://www.ac.tuwien.ac.at/people/szeider/cartoon/"><img src ="npc_cartoon3.png" width=70%></a>



### Parallelism? ###

Can we parallelize and solve $\mathcal{NP}$-complete problems? 

Since the definition of span doesn't really care about the number of processors, we can solve problems in $\mathcal{NP}$ using brute force with polynomial span. This is because the definition of $\mathcal{NP}$ ensures that we can efficiently check candidate solutions.

However if we were able to achieve polynomial work, we'd immediately have shown $\mathcal{P}=\mathcal{NP}$ since we could just do all the work on a single processor.

A more interesting question is whether we can effectively parallelize problems in $\mathcal{P}$. That is, for any problem $\mathcal{X}$ that is solvable in polynomial work, does it also have low span?

Let $\mathcal{NC}$ denote the set of all problems with span $O(\log^c n)$ for some constant $c$ using a polynomial number of processors. 

We know that $\mathcal{NC}\subseteq \mathcal{P}$, but is $\mathcal{P}\subseteq\mathcal{NC}$? 

What does this statement imply?

It implies that **every** problem in $\mathcal{P}$ is parallelizable. 

As with $\mathcal{NP}$, it's possible to define $\mathcal{P}$-complete problems. 

The Circuit Value Problem ($\mathit{CVP}$) asks, for a given circuit on AND, OR, and NOT gates along with inputs, does the circuit produce an output of 1?

$\mathit{CVP}$ is $\mathcal{P}$-complete.

As with $\mathcal{NP}$, we have [a long list of $\mathcal{P}$-complete problems](https://en.wikipedia.org/wiki/P-complete#P-complete_problems). We cannot find polylogarithmic span algorithms for any of them, nor can we prove that it is not possible to come up with such algorithms.

We have just looked at two complexity classes, but this <a href="https://complexityzoo.uwaterloo.ca/Complexity_Zoo">area</a> is quite deep.


<a href = "https://jeremykun.com/2012/02/29/other-complexity-classes/"><img src="complexity_venn_diagram.jpg" width=60%></a>